In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Lung Cancer Detection Model Training\n",
    "\n",
    "This notebook contains the complete pipeline for training a deep learning model to detect lung cancer from medical images."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import necessary libraries\n",
    "import os\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras import layers, models\n",
    "from tensorflow.keras.preprocessing.image import ImageDataGenerator\n",
    "from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Data Loading and Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Set up data directories\n",
    "data_dir = 'Lung Cancer Dataset/lung_colon_image_set'\n",
    "img_height, img_width = 224, 224\n",
    "batch_size = 32\n",
    "\n",
    "# Data augmentation for training\n",
    "train_datagen = ImageDataGenerator(\n",
    "    rescale=1./255,\n",
    "    rotation_range=20,\n",
    "    width_shift_range=0.2,\n",
    "    height_shift_range=0.2,\n",
    "    horizontal_flip=True,\n",
    "    fill_mode='nearest',\n",
    "    validation_split=0.2\n",
    ")\n",
    "\n",
    "# Only rescaling for validation\n",
    "val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)\n",
    "\n",
    "# Load training data\n",
    "train_generator = train_datagen.flow_from_directory(\n",
    "    data_dir,\n",
    "    target_size=(img_height, img_width),\n",
    "    batch_size=batch_size,\n",
    "    class_mode='binary',\n",
    "    subset='training'\n",
    ")\n",
    "\n",
    "# Load validation data\n",
    "validation_generator = val_datagen.flow_from_directory(\n",
    "    data_dir,\n",
    "    target_size=(img_height, img_width),\n",
    "    batch_size=batch_size,\n",
    "    class_mode='binary',\n",
    "    subset='validation'\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Model Architecture"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def create_model():\n",
    "    model = models.Sequential([\n",
    "        # First Convolutional Block\n",
    "        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),\n",
    "        layers.MaxPooling2D((2, 2)),\n",
    "        \n",
    "        # Second Convolutional Block\n",
    "        layers.Conv2D(64, (3, 3), activation='relu'),\n",
    "        layers.MaxPooling2D((2, 2)),\n",
    "        \n",
    "        # Third Convolutional Block\n",
    "        layers.Conv2D(128, (3, 3), activation='relu'),\n",
    "        layers.MaxPooling2D((2, 2)),\n",
    "        \n",
    "        # Fourth Convolutional Block\n",
    "        layers.Conv2D(128, (3, 3), activation='relu'),\n",
    "        layers.MaxPooling2D((2, 2)),\n",
    "        \n",
    "        # Flatten and Dense Layers\n",
    "        layers.Flatten(),\n",
    "        layers.Dropout(0.5),\n",
    "        layers.Dense(512, activation='relu'),\n",
    "        layers.Dropout(0.5),\n",
    "        layers.Dense(1, activation='sigmoid')\n",
    "    ])\n",
    "    \n",
    "    return model\n",
    "\n",
    "# Create and compile the model\n",
    "model = create_model()\n",
    "model.compile(\n",
    "    optimizer='adam',\n",
    "    loss='binary_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "# Display model summary\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Training Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Define callbacks\n",
    "callbacks = [\n",
    "    EarlyStopping(\n",
    "        monitor='val_loss',\n",
    "        patience=10,\n",
    "        restore_best_weights=True\n",
    "    ),\n",
    "    ModelCheckpoint(\n",
    "        'best_model.h5',\n",
    "        monitor='val_accuracy',\n",
    "        save_best_only=True\n",
    "    ),\n",
    "    ReduceLROnPlateau(\n",
    "        monitor='val_loss',\n",
    "        factor=0.2,\n",
    "        patience=5,\n",
    "        min_lr=1e-6\n",
    "    )\n",
    "]\n",
    "\n",
    "# Train the model\n",
    "history = model.fit(\n",
    "    train_generator,\n",
    "    epochs=50,\n",
    "    validation_data=validation_generator,\n",
    "    callbacks=callbacks\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Model Evaluation and Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot training history\n",
    "def plot_training_history(history):\n",
    "    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n",
    "    \n",
    "    # Plot accuracy\n",
    "    ax1.plot(history.history['accuracy'], label='Training Accuracy')\n",
    "    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "    ax1.set_title('Model Accuracy')\n",
    "    ax1.set_xlabel('Epoch')\n",
    "    ax1.set_ylabel('Accuracy')\n",
    "    ax1.legend()\n",
    "    \n",
    "    # Plot loss\n",
    "    ax2.plot(history.history['loss'], label='Training Loss')\n",
    "    ax2.plot(history.history['val_loss'], label='Validation Loss')\n",
    "    ax2.set_title('Model Loss')\n",
    "    ax2.set_xlabel('Epoch')\n",
    "    ax2.set_ylabel('Loss')\n",
    "    ax2.legend()\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# Plot the training history\n",
    "plot_training_history(history)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Evaluate the model on validation data\n",
    "val_loss, val_accuracy = model.evaluate(validation_generator)\n",
    "print(f'\\nValidation Accuracy: {val_accuracy:.4f}')\n",
    "print(f'Validation Loss: {val_loss:.4f}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get predictions\n",
    "predictions = model.predict(validation_generator)\n",
    "y_pred = (predictions > 0.5).astype(int)\n",
    "y_true = validation_generator.classes\n",
    "\n",
    "# Plot confusion matrix\n",
    "cm = confusion_matrix(y_true, y_pred)\n",
    "plt.figure(figsize=(8, 6))\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')\n",
    "plt.title('Confusion Matrix')\n",
    "plt.ylabel('True Label')\n",
    "plt.xlabel('Predicted Label')\n",
    "plt.show()\n",
    "\n",
    "# Print classification report\n",
    "print('\\nClassification Report:')\n",
    "print(classification_report(y_true, y_pred))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined